In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [5]:
class Network(nn.Module):
    def __init__(self, hidden_layers, hidden_layer_neurons):
        super(Network, self).__init__()
        self.layers = [nn.Linear(1,hidden_layer_neurons)]
        for _ in range(hidden_layers):
            self.layers.append(nn.Linear(hidden_layer_neurons, hidden_layer_neurons))
        self.layers.append(nn.Linear(hidden_layer_neurons, 1))

    def forward(self, x):
        inputs = torch.cat([x],axis=1)
        out_layers = [torch.sigmoid(self.layers[0](inputs))]
        for i in range(1,len(self.layers)-1):
            out_layers.append(torch.sigmoid(self.layers[i](out_layers[i-1])))
        output = self.layers[-1](out_layers[-1])
        return output

